In [1]:
import os
os.chdir('/Users/akshit.jain/repos/commoncrawl/')

In [64]:
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import StemmingAnalyzer,StandardAnalyzer
import os, os.path
from whoosh import index
from whoosh import writing
from whoosh.writing import BufferedWriter, AsyncWriter
from whoosh.qparser import QueryParser
from whoosh.query import Phrase, And,Term
from IPython.core.display import display, HTML

import pandas as pd


In [4]:
mc=pd.read_pickle('./data/indian_movies_cast.pk')

In [7]:
ar=pd.read_pickle('/Users/akshit.jain/repos/commoncrawl/output/parsed_pages.pk')
ar=ar.meta

In [96]:
schema = Schema(body=TEXT(analyzer=StandardAnalyzer(stoplist=None), stored=True),
                url=ID(stored=True), id=ID(stored=True))


if not os.path.exists("indexdir"):
    os.mkdir("indexdir")

ix = index.create_in("indexdir", schema)

In [97]:
with AsyncWriter(ix) as writer:
    try:
        writer.mergetype = writing.CLEAR
        def add_doc(article, id):
            writer.add_document(body=article.story, url=article.url, id=str(id))
        for idx, article in ar.iteritems():
            if idx%100==0:
                print('#{}'.format(idx))
            add_doc(article, idx)
        writer.commit()
    finally:
        pass
        #writer.close()

#0
#100
#200
#300
#400
#500
#600
#700
#800
#900
#1000
#1100
#1200
#1300
#1400
#1500
#1600
#1700
#1800
#1900
#2000
#2100
#2200
#2300
#2400
#2500
#2600
#2700
#2800
#2900
#3000


IndexingError: This writer is closed

In [98]:
actor_occ=mc.groupby('actor').count().reset_index()
popular_actors=actor_occ[actor_occ.movie>=10].sort_values('movie', ascending=False)
popular_mc=mc[mc.actor.isin(popular_actors.actor)]

In [117]:
import re
from nltk.tokenize import sent_tokenize


qp = QueryParser("body", schema=ix.schema)
s=ix.searcher()

def get_matches(phrase, corpus):
    sents = sent_tokenize(corpus)
    for sent in sents:
        matches = [m.start() for m in re.finditer(phrase.lower(), sent.lower())]
        matches = [(start, start+len(phrase)) for start in matches]
        if len(matches)>0:
            yield {'sentence': sent, 'matches': matches}
def sentence_matches_from_hits(phrase, hits):
    for hit in hits:
        yield from get_matches(phrase, hit['body']) 


def actor_movie_hits(row):
    if row.name%1000==0:
        print('#{}'.format(row.name))
    q = qp.parse(u'"{}" AND "{}"'.format(row.actor, row.movie))
    hits = s.search(q)
    return list(sentence_matches_from_hits(row.movie, hits))
_mc=popular_mc.reset_index()
_mc['hits']=_mc.apply(actor_movie_hits, axis=1)

#0
#1000
#2000
#3000
#4000
#5000
#6000
#7000
#8000
#9000
#10000
#11000
#12000
#13000
#14000
#15000
#16000
#17000
#18000
#19000
#20000
#21000
#22000
#23000
#24000
#25000
#26000
#27000
#28000
#29000
#30000
#31000
#32000
#33000
#34000
#35000
#36000
#37000
#38000
#39000
#40000
#41000
#42000
#43000
#44000
#45000
#46000
#47000
#48000
#49000
#50000
#51000
#52000
#53000
#54000
#55000
#56000
#57000
#58000
#59000
#60000
#61000
#62000
#63000
#64000
#65000
#66000
#67000
#68000
#69000
#70000
#71000
#72000
#73000
#74000
#75000
#76000
#77000
#78000
#79000
#80000
#81000
#82000
#83000
#84000
#85000
#86000
#87000
#88000
#89000
#90000
#91000
#92000
#93000
#94000
#95000
#96000
#97000
#98000
#99000
#100000
#101000
#102000
#103000
#104000
#105000
#106000
#107000
#108000
#109000
#110000
#111000
#112000
#113000
#114000
#115000
#116000
#117000
#118000
#119000
#120000
#121000
#122000
#123000
#124000
#125000
#126000
#127000
#128000
#129000
#130000
#131000
#132000
#133000
#134000
#135000
#136000
#137000
#138000
#

In [106]:
    _mc['mentions']=_mc.apply(lambda row: list(sentence_matches_from_hits(row.movie, row.hits)), axis=1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [120]:
_mc['num_hits']=_mc.hits.apply(len)
_mc[_mc.num_hits>0].head()
_mc=_mc[_mc.num_hits>0]

In [145]:
# remove movies made of only stop words
from nltk.corpus import stopwords
stoplist=frozenset(stopwords.words('english'))
_mc=_mc[~_mc.movie.apply(lambda x: x.lower() in stoplist)]
# _mc.sort_values('num_hits', ascending=False)

In [146]:
all_mentions=[]
def collect_mentions(x):
    global all_mentions
    all_mentions.extend(x)
_mc.hits.apply(collect_mentions)
all_mentions[:10]

[{'sentence': 'Anoushka has given the background score for the 1928 silent film, "Shiraz: a Romance of India", which was resurrected by BFI London.',
  'matches': [(67, 73)]},
 {'sentence': "'Shiraz' is an important part of our filmmaking and cultural history.",
  'matches': [(1, 7)]},
 {'sentence': 'But that affected the background score of "Shiraz" initially.',
  'matches': [(43, 49)]},
 {'sentence': 'Post your comments  \n\n Anoushka Shankaranoushka shankar newsAnoushka Shankar Tourshiraz',
  'matches': [(81, 87)]},
 {'sentence': '#Temptationtour #Toomuchfun #ThrowbackThursday #Madness #Happiness #Friendship #Ting😘\u202c" \n  Credit: @Preity Zinta  \n\n \r\n The black and white photo sure brings back the fond memories from 2004 when Preity, Rani and Saif were every filmmaker\'s favourite post the success of  Kal Ho Na Ho and Hum Tum, while Priyanka and Arjun were finding their mark in the industry.',
  'matches': [(124, 139)]},
 {'sentence': '#GandhiJayanti— Madhuri Dixit Nene (@Mad

In [147]:
mentions=pd.DataFrame(all_mentions)

In [148]:
def union_mentions(matches):
    match_set=set()
    for match_list in matches:
        match_set = match_set | frozenset(match_list)
    return list(match_set)
movie_corpus=mentions.groupby('sentence').apply(lambda x:  union_mentions(x.matches)).to_frame('spans').reset_index()

In [153]:
movie_corpus['num_spans']=movie_corpus['spans'].apply(len)
movie_corpus.sort_values('num_spans', ascending=False).head()

,sentence,spans,num_spans
1318,Actor Tiger Shroff grooved to hits like 'The P...,"[(1750, 1756), (825, 836), (1463, 1481), (1319...",20
62,\nBest Director- Vikas Bahl for 'Queen' Best ...,"[(355, 367), (1494, 1506), (52, 57), (197, 203...",18
2593,Here is the Complete list of National Award 20...,"[(2034, 2039), (2131, 2143), (2318, 2332), (23...",15
2537,He then starred in successful films such as Ji...,"[(124, 133), (51, 58), (165, 172), (60, 66), (...",11
4901,"The story is of Bitti who lives in Bareilly, b...","[(65, 66), (193, 194), (26, 27), (90, 91), (93...",11


In [154]:
movie_corpus.to_pickle('data/sentence_movie_occ_df.pk')

In [163]:
movie_corpus=movie_corpus.sample(frac=1.0)

In [73]:
display(HTML(hit.highlights('body',top=3,)))

# re-train spacy ER model

In [164]:
TRAIN_DATA=[(x.sentence, {'entities': [(el[0], el[1], 'MOVIE') for el in x.spans]}) for idx,x in movie_corpus.iterrows()]

In [165]:
len(TRAIN_DATA)

5770

In [169]:
import pickle as pk
with open('data/spacy_movie_ner_train.pk','wb') as f:
    pk.dump(TRAIN_DATA, f)

In [172]:
# ! scp data/spacy_movie_ner_train.pk gpu-sd:/mnt/data/akshit.jain/kg/data/
! scp ./train_new_entity_type.py gpu-sd:/mnt/data/akshit.jain/kg/

train_new_entity_type.py                      100% 4335     8.2KB/s   00:00    
Killed by signal 1.


In [167]:
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


# new entity label
LABEL = 'MOVIE'

# training data
# Note: If you're using an existing model, make sure to mix in examples of
# other entity types that spaCy correctly recognized before. Otherwise, your
# model might learn the new type, but "forget" what it previously knew.
# https://explosion.ai/blog/pseudo-rehearsal-catastrophic-forgetting
def main(model='en_core_web_md', new_model_name='movie_ner', output_dir='data/model', n_iter=10):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")
    # Add entity recognizer to model if it's not in the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe('ner')

    ner.add_label(LABEL)   # add new entity label to entity recognizer
    if model is None:
        optimizer = nlp.begin_training()
    else:
        # Note that 'begin_training' initializes the models, so it'll zero out
        # existing entity types.
        optimizer = nlp.entity.create_optimizer()

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35,
                           losses=losses)
            print('Losses', losses)

    # test the trained model
    test_text = 'Amir Khan gave a splendid performance in Dhoom 3'
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta['name'] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)

In [ ]:
main()

In [200]:
doc, ents=TRAIN_DATA[0]
import re
def pre_process(doc):
    al_pattern=r'([a-zA-Z]+)'
    num_pattern=r'([\d]+)'
    doc=re.sub(al_pattern, r' \1 ', doc)
    doc=re.sub(num_pattern, r' \1 ', doc)
    doc=re.sub('\s+',' ', doc)
    return doc.split()
pre_process(doc)

['For',
 'a',
 'story',
 'that',
 'unfolds',
 'through',
 'the',
 'course',
 'of',
 'a',
 'single',
 'night',
 ',',
 '96',
 'feels',
 'long',
 'but',
 'it',
 "'",
 's',
 'for',
 'its',
 'own',
 'good',
 '.']

In [236]:
# class Token():
#     token=''
#     tag=''

def parse(doc, ents):
    
    ents=ents['entities']
    entities=[]
    for start,end, _ in ents:
        entity=doc[start: end]
        entity=re.sub('\W',' ', entity)
        entity = pre_process(entity)
        entities.append(entity)
    doc=pre_process(doc)
    tags=['O']*len(doc)
    for idx, tok in enumerate(doc):
        for entity in entities:
            if ' '.join([w.lower() for w in doc[idx: idx+len(entity)]])==' '.join([e.lower() for e in entity]):
                # print('{} matched'.format(' '.join(entity)))
                tags[idx: idx+len(entity)]=['M']*len(entity)
    return doc, tags

train_data_parsed=[]
for doc, ents in TRAIN_DATA:
    doc_tokens, tags=parse(doc, ents)
    assert len(doc_tokens)==len(tags)
    corpus=list(zip(doc_tokens, tags))
    train_data_parsed.append(corpus)

# import pickle as pk
# with open('data/bert_ner_train_data.pk', 'wb') as f:
#     pk.dump(train_data_parsed, f)


In [237]:
import random

NUM_SAMPLES=len(train_data_parsed)
NUM_DEV=int(0.1*NUM_SAMPLES)
NUM_TRAIN=NUM_SAMPLES-NUM_DEV

train_data=train_data_parsed[0: NUM_TRAIN]
dev_data=train_data_parsed[NUM_TRAIN:]

import pickle as pk
DATA_DUMP_DIR='/Users/akshit.jain/repos/BERT-NER/MovNERdata'
with open(os.path.join(DATA_DUMP_DIR,'bert_ner_train_data.pk'), 'wb') as f:
    pk.dump(train_data, f)

with open(os.path.join(DATA_DUMP_DIR,'bert_ner_dev_data.pk'), 'wb') as f:
    pk.dump(dev_data, f)


In [226]:
NUM_SAMPLES

5770

In [239]:
for dat in train_data:
    print(dat)
    break

[('For', 'O'), ('a', 'O'), ('story', 'O'), ('that', 'O'), ('unfolds', 'O'), ('through', 'O'), ('the', 'O'), ('course', 'O'), ('of', 'O'), ('a', 'O'), ('single', 'O'), ('night', 'O'), (',', 'O'), ('96', 'M'), ('feels', 'O'), ('long', 'O'), ('but', 'O'), ('it', 'O'), ("'", 'O'), ('s', 'O'), ('for', 'O'), ('its', 'O'), ('own', 'O'), ('good', 'O'), ('.', 'O')]


In [250]:
len(dev_data[0])

34

In [255]:
with open('/Users/akshit.jain/repos/BERT-NER/output/predict_results.pk', 'rb') as f:
    preds = pk.load(f)

In [287]:
import numpy as np
import re
def extract_entity(tokens, labels):
    movie_mask=np.array([False]*len(labels))
    for idx, lbl in enumerate(labels):
        if labels[idx]==2 or (movie_mask[idx-1] and (tokens[idx][:2]=='##')):
            movie_mask[idx]=True    
    movie_mask=movie_mask[:len(tokens)]
    return re.sub(r' ##', '',' '.join(list(np.array(tokens)[movie_mask])))
for p in preds[100:200]:
    print(extract_entity(p['tokens'], p['labels_true']))
    print(extract_entity(p['tokens'], p['labels_pred']))
    print('-'*10)

pataakha
pataakha
----------
sanju
sanju
----------
kabuliwala
kabuliwala
----------
wedding
wedding
----------
english vinglish
english vinglish
----------
go housefull

----------
helicopter eela
helicopter eela
----------
fanne khan
fanne khan taal
----------

black
----------
dus ka dum
dus ka dum
----------
good news
good news
----------
manto
manto mantonandita
----------
sultan bajrangi bhaijaan
sultan bajrangi bhaijaan
----------
i am
i am
----------
vip
hero vip 2
----------
anbe sivam
anbe sivam
----------
call

----------
sethupathi
sir vijay sethupathi
----------


----------
sunny
sunny
----------
tumhari sulu kahaani hichki
tumhari sulu kahaani hichki mardani
----------


----------
plan
plan
----------
nagavalli a aa
a aa
----------
shani
har har mahadev
----------
manto
manto
----------
secret superstar
secret superstar
----------
vip vip
vip 2 vip 2
----------
nh 10
nh 10
----------
raazi
raazi
----------


----------
queen
queen
----------


----------
company
show dr

In [246]:
pred_res=[[int(x) for x in line.split()] for line in lines]

In [249]:
len(pred_res[0])

128

In [ ]:
convert_single_example(20, )

In [254]:
list([1,3])

[1, 3]